In [ ]:
%matplotlib inline
from VisualPortfolio import createPerformanceTearSheet
from DataAPI import api

1. DataAPI与VisualPortfolio结合使用
=========================
***

> 我们可以使用DataAPI获取某只证券的价格数据，简详细的绘制它的表现：

In [ ]:
sample_prices = api.GetEquityBarMin1('600000', '2012-10-01', '2015-11-09')

In [ ]:
sample_prices.tail()

In [ ]:
sample_prices[['closePrice']].plot(figsize=(14,8))

1.1 收益表现
---------------

* ``prices``：需要进行分析的价格序列；
* ``benchmark``：与之进行对比的指数；

In [ ]:
perf_metric, perf_df = createPerformanceTearSheet(prices=sample_prices['closePrice'], benchmark='000300.zicn')

2. 证券头寸变化分析
-----------------------
***

In [ ]:
import pandas as pd
from VisualPortfolio import createPostionTearSheet

我们可以读入一个我们已有的position book（表中的数据都是改证券当日净值）：

In [ ]:
positions = pd.read_csv('data/positions.csv', index_col=0, parse_dates=[0])

In [ ]:
positions[50:60]

绘制头寸分析图：

In [ ]:
positions = createPostionTearSheet(positions)

当然这个内容还需要补充。

3. 交易行为分析
==================
***

In [ ]:
from VisualPortfolio import createTranscationTearSheet
transactions = pd.read_csv('data/transactions.csv', index_col=0, parse_dates=[0])

In [ ]:
transactions.head()

In [ ]:
turnover_rate = createTranscationTearSheet(positions=positions, transactions=transactions)

In [ ]:
turnover_rate.tail()

4. 所有的都已经整合入``AlgoTrading``
=======================
***

In [ ]:
import datetime as dt

from AlgoTrading.Strategy.Strategy import Strategy
from AlgoTrading.Backtest import strategyRunner
from AlgoTrading.Backtest import DataSource
from AlgoTrading.Data import set_universe
from PyFin.API import MA
from PyFin.API import MAX
from PyFin.API import MIN


class MovingAverageCrossStrategy(Strategy):
    def __init__(self):
        filtering = (MAX(10, 'close') / MIN(10, 'close')) > 1.00
        indicator = MA(10, 'close') - MA(120, 'close')
        self.signal = indicator[filtering]

    def handle_data(self):
        for s in self.universe:
            amount = self.avaliableForSale(s)
            if self.signal[s] > 0. and self.secPos[s] == 0:
                self.order(s, 1, quantity=100)
            elif self.signal[s] < 0. and amount != 0:
                self.order(s, -1, quantity=amount)


def run_example():
    universe = set_universe('000300.zicn')
    initialCapital = 100000.0
    startDate = dt.datetime(2006, 10, 1)
    endDate = dt.datetime(2015, 10, 1)

    strategyRunner(userStrategy=MovingAverageCrossStrategy,
                   initialCapital=initialCapital,
                   symbolList=universe,
                   startDate=startDate,
                   endDate=endDate,
                   dataSource=DataSource.DataYes,
                   benchmark='000300.zicn',
                   logLevel='critical',
                   saveFile=True,
                   plot=True)

In [ ]:
startTime = dt.datetime.now()
print("Start: %s" % startTime)
res = run_example()
endTime = dt.datetime.now()
print("End : %s" % endTime)
print("Elapsed: %s" % (endTime - startTime))